In [7]:
!pip install --upgrade openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client plyer


  Using cached openai-1.97.1-py3-none-any.whl.metadata (29 kB)
Using cached openai-1.97.1-py3-none-any.whl (764 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 1.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: openai━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/14 [oauthlib]
    Found existing installation: openai 1.95.1━━━━━━━━━━━━━━━━  4/14 [oauthlib]
    Uninstalling openai-1.95.1:━━╸━━━━━━━━━━━━━━  9/14 [openai]
      Successfully uninstalled openai-1.95.1━━━━━━━━━━━━━━  9/14 [openai]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [google-api-python-client]api-python-client]


In [35]:

import os
import pickle
import base64
from email.mime.text import MIMEText
from dotenv import load_dotenv
from openai import OpenAI
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
if os.path.exists("token.pkl"):
    os.remove("token.pkl")
# 🔐 Load OpenAI key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 🔑 Gmail Auth with modify permission
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
service = authenticate_gmail()




Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=751782088568-jfkr8uha4kfok3s0sdfndhe4ghds3juj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57273%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=VWbgbw2nQgJwjYZdhld4GHJzT3JuPn&access_type=offline


In [36]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os, pickle

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pickle.load(token)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.pkl', 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)


In [42]:
def get_emails(service, max_results=5):
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], maxResults=max_results).execute()
    messages = results.get('messages', [])
    
    email_data = []
    for msg in messages:
        msg_detail = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
        headers = msg_detail['payload']['headers']
        
        email_info = {
            'snippet': msg_detail.get('snippet', ''),
            'to': next((h['value'] for h in headers if h['name'] == 'From'), None),
            'subject': next((h['value'] for h in headers if h['name'] == 'Subject'), ''),
            'msg_id': next((h['value'] for h in headers if h['name'] == 'Message-ID'), None),
        }
        email_data.append(email_info)
    
    return email_data


In [38]:
# 🧠 Classify & reply using OpenAI
def classify_and_respond(email_text):
    prompt = f"""You are an intelligent email assistant. 
Classify the following email as one of: urgent, spam, or follow-up.
Then generate a short, polite response.

Email: {email_text}

Respond in this format:
Classification: <category>
Reply: <your reply>
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )

    result = response.choices[0].message.content
    classification = None
    reply = None

    for line in result.splitlines():
        if line.lower().startswith("classification:"):
            classification = line.split(":", 1)[1].strip().lower()
        elif line.lower().startswith("reply:"):
            reply = line.split(":", 1)[1].strip()

    return classification, reply

In [44]:
from email.mime.text import MIMEText
import base64

def create_draft(service, to, subject, reply_text, in_reply_to=None):
    message = MIMEText(reply_text)
    message['to'] = to
    message['subject'] = "Re: " + subject
    if in_reply_to:
        message['In-Reply-To'] = in_reply_to

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    draft_body = {'message': {'raw': raw}}
    
    draft = service.users().drafts().create(userId='me', body=draft_body).execute()
    print(f"✅ Draft created for: {to}")


In [47]:
service = authenticate_gmail()
emails = get_emails(service)

for email in emails:
    classification, reply = classify_and_respond(email.get("snippet", ""))

    print("---------")
    print("Original:", email.get("snippet", ""))
    print("Classification:", classification)
    print("Reply:", reply)

    if classification != "spam" and reply:
        create_draft(
            service,
            to=email.get("to"),
            subject=email.get("subject"),
            reply_text=reply,
            in_reply_to=email.get("msg_id")
        )
    else:
        print("🚫 Skipped (spam or no reply)")


---------
Original: Hello Divanshu .
Classification: follow-up
Reply: Hello, thank you for reaching out. Could you please provide more details or context regarding your message? Looking forward to your response.
✅ Draft created for: divanshu n8n <divanshunn@gmail.com>
---------
Original: Join us for the interviews, gameplay reveals and more ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌
Classification: spam
Reply: Thank you for reaching out. I am currently not interested in this offer, but I appreciate the invitation.
🚫 Skipped (spam or no reply)
---------
Original: Affordable degrees from top global universities ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌
Classification: spam
Reply: Thank you for reaching out. Please note that I am not interested in this offer at the moment.
🚫 

In [48]:
from plyer import notification

notification.notify(
    title='📬 Email Classifier',
    message='✔️ All top 5 emails checked. Replies drafted (if needed).',
    timeout=6
)
